In [3]:
pip install Pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.5 MB 5.6 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.5 MB 6.1 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/11.5 MB 5.9 MB/s eta 0:00:02
   ---------------- ----------------------- 4.7/11.5 MB 5.9 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.5 MB 5.7 MB/s eta 0:00:02
   ----------------------- ---------------- 6.8/11.5 MB 5.8 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.5 MB 5.8 MB/s eta 0:00:01
   ------------------------------- -------- 9.2/11.5 MB 5.8 MB/s eta 0:00:01
   ------------------------------------ --- 10.5/11.5 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
